# Notice

All demos have been moved to the `/demos` directory in the root of the project [View on GitHub](https://github.com/TransformerLensOrg/TransformerLens/tree/main/demos)


In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='4'
import sys
sys.path.append('/home/kitsuchart/aakash/deep_research/TransformerLens/')
from transformer_lens import *

In [2]:
device = utils.get_device()

# Load the model
model= HookedTransformer.from_pretrained("smol-500M", device)

Loaded pretrained model smol-500M into HookedTransformer


In [3]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(0)

processor = AutoProcessor.from_pretrained(model_id)

# Define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "What are these?"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

image_file = "http://images.cocodataset.org/val2017/000000039769.jpg"
raw_image = Image.open(requests.get(image_file, stream=True).raw)
# inputs = processor(images=raw_image, text=prompt, return_tensors='pt').to(0, torch.float16)
inputs = processor(images=raw_image, text=prompt, return_tensors='pt').to(torch.float16)

# output = model.generate(**inputs, max_new_tokens=200, do_sample=False)
# print(processor.decode(output[0][2:], skip_special_tokens=True))


In [4]:
# vit_embed = vision_model.to('cuda').get_image_features(
#                 pixel_values=inputs.pixel_values
#             ) 
vit_embed = vision_model.get_image_features(
                pixel_values=inputs.pixel_values
            ) 
# image_features = torch.cat(vit_embed, dim=0)

In [5]:
# vit_embed.shape
model

In [6]:
# model_description_text = """## Loading Models

# HookedTransformer comes loaded with >40 open source GPT-style models. You can load any of them in with `HookedTransformer.from_pretrained(MODEL_NAME)`. See my explainer for documentation of all supported models, and this table for hyper-parameters and the name used to load them. Each model is loaded into the consistent HookedTransformer architecture, designed to be clean, consistent and interpretability-friendly. 

# For this demo notebook we'll look at GPT-2 Small, an 80M parameter model. To try the model the model out, let's find the loss on this paragraph!"""
# # model_description_text=  processor.decode(inputs.input_ids[0][1:])
# loss = model(model_description_text, return_type="loss")
# print("Model loss:", loss)

AssertionError: 

In [ ]:
llava_text = "Natural language processing tasks, such as question answering, machine translation, reading comprehension, and summarization, are typically approached with supervised learning on taskspecific datasets."
# llava_text = processor.decode(inputs.input_ids[0][1:])
llava_tokens = model.to_tokens(llava_text)
print(llava_tokens.device)
llava_logits, llava_cache = model.run_with_cache(llava_tokens, remove_batch_dim=True)

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
import plotly.express as px

from jaxtyping import Float
from functools import partial


def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)


In [ ]:
import circuitsvis as cv
# Testing that the library works

In [ ]:
print(type(llava_cache))
attention_pattern = llava_cache["pattern", 0, "attn"]
print(attention_pattern.shape)
llava_str_tokens = model.to_str_tokens(llava_text)

In [ ]:
print("Layer 0 Head Attention Patterns:")
cv.attention.attention_patterns(tokens=llava_str_tokens, attention=attention_pattern)

In [ ]:
# processor.decode(inputs.input_ids[0][1:])

In [ ]:
# residual,tokens,shortformer_pos_embed,attention_mask = model.input_to_embed(
#                    llava_text
#                 )

In [ ]:
# special_image_mask = (llava_tokens == vision_model.config.image_token_id).unsqueeze(-1)
# special_image_mask = special_image_mask.expand_as(residual).to(residual.device)

In [ ]:
# vision_model.config.image_token_id

In [ ]:
# image_features = vit_embed.to(residual.device, residual.dtype)
# inputs_embeds = residual.masked_scatter(special_image_mask, image_features)

In [ ]:
# inputs_embeds.shape,residual.shape,special_image_mask.shape

In [ ]:
# ans = inputs_embeds==residual

In [ ]:
# ans.shape

In [ ]:
# ans[0][10]

In [ ]:
# # Load model directly
# from transformers import AutoProcessor, AutoModelForImageTextToText

# processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
# model = AutoModelForImageTextToText.from_pretrained("llava-hf/llava-1.5-7b-hf", device_map='auto')

In [ ]:
# model.config

In [ ]:
# model.language_model.layers[0].input_layernorm.weight.shape